Get TCP and UDP

In [39]:
import psutil
import subprocess
import csv
from tabulate import tabulate
import tkinter as tk
from tkinter import ttk

# Função auxiliar para identificar processos svchost e pegar os serviços
def get_svchost_services(pid):
    try:
        output = subprocess.check_output(
            ["powershell", "-Command",
             f"Get-WmiObject -Class Win32_Service | Where-Object {{$_.ProcessId -eq {pid}}} | Select-Object -ExpandProperty Name"],
            universal_newlines=True
        )
        return ", ".join([line.strip() for line in output.strip().splitlines() if line.strip()])
    except Exception:
        return "Nenhum serviço associado"

# Coleta TCP Connections
def get_tcp_connections():
    data = []
    for conn in psutil.net_connections(kind='tcp'):
        try:
            proc = psutil.Process(conn.pid) if conn.pid else None
            pname = proc.name() if proc else "N/A"
            services = get_svchost_services(conn.pid) if pname.lower() == "svchost.exe" else "Nenhum serviço associado"

            row = [
                conn.laddr.ip if conn.laddr else '',
                conn.laddr.port if conn.laddr else '',
                conn.raddr.ip if conn.raddr else '',
                conn.raddr.port if conn.raddr else '',
                conn.pid,
                pname,
                services,
                conn.status
            ]
            data.append(row)
        except Exception:
            continue
    return data

# Coleta UDP Endpoints
def get_udp_connections():
    data = []
    for conn in psutil.net_connections(kind='udp'):
        try:
            proc = psutil.Process(conn.pid) if conn.pid else None
            pname = proc.name() if proc else "Desconhecido/Finalizado"
            services = get_svchost_services(conn.pid) if pname.lower() == "svchost.exe" else "Nenhum serviço associado"

            row = [
                conn.laddr.ip if conn.laddr else '',
                conn.laddr.port if conn.laddr else '',
                '',
                '',
                conn.pid,
                pname,
                services,
                'UDP'
            ]
            data.append(row)
        except Exception:
            continue
    return data

# Exporta para CSV
def export_csv(data, filename):
    headers = ["LocalAddress", "LocalPort", "RemoteAddress", "RemotePort", "ProcessId", "ProcessName", "Services", "State"]
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(data)

# Função para exibir as informações de rede no formato tabular
def display_connections():
    tcp_data = get_tcp_connections()
    udp_data = get_udp_connections()
    full_data = tcp_data + udp_data

    # Exibe as conexões no formato de tabela
    return full_data

# Função para criar a tabela com Tkinter
def create_table(window, data, headers):
    frame = ttk.Frame(window)
    frame.pack(fill='both', expand=True)

    tree = ttk.Treeview(frame, columns=headers, show='headings')
    
    # Configura cabeçalhos das colunas
    for col in headers:
        tree.heading(col, text=col)
        tree.column(col, width=150, anchor='w')
    
    # Preenche a tabela com os dados
    for row in data:
        tree.insert("", "end", values=row)

    tree.pack(fill='both', expand=True)

# Função para abrir a janela e exibir as conexões no Tkinter
def show_connections():
    full_data = display_connections()
    headers = ["LocalAddress", "LocalPort", "RemoteAddress", "RemotePort", "ProcessId", "ProcessName", "Services", "State"]

    # Criação da janela principal do Tkinter
    root = tk.Tk()
    root.title("Monitoramento de Conexões TCP/UDP")
    root.geometry("1000x600")

    # Título da janela
    tk.Label(root, text="Conexões TCP/UDP Ativas", font=("Arial", 16, "bold")).pack(pady=10)

    # Criação da tabela com os dados coletados
    create_table(root, full_data, headers)

    # Exibição da janela
    root.mainloop()

# Main
if __name__ == "__main__":
    # Exibe as conexões no terminal
    full_data = display_connections()

    # Exporta para CSV
    export_csv(full_data, "network_connections.csv")

    # Exibe a interface Tkinter
    show_connections()


Verificação Whois e Virustotal (psutil)

In [63]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import psutil
import requests
from ipwhois import IPWhois
import os
from datetime import datetime

# Configurações da API do VirusTotal
VT_API_KEY = "5604dd90f9a804bfcfca0bfd4dce40f51687c8c1207086429e31a044df7bc654"
VT_URL = "https://www.virustotal.com/api/v3/ip_addresses/"
HEADERS = {"x-apikey": VT_API_KEY}

# Cria pasta de logs
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
os.makedirs("logs", exist_ok=True)

# Coleta conexões
connections = []
for conn in psutil.net_connections(kind='inet'):
    if conn.raddr:
        try:
            proc = psutil.Process(conn.pid) if conn.pid else None
            process_name = proc.name() if proc else "Desconhecido"
        except:
            process_name = "Erro"
        connections.append({
            "Protocolo": conn.type,
            "LocalAddress": f"{conn.laddr.ip}:{conn.laddr.port}",
            "RemoteAddress": f"{conn.raddr.ip}:{conn.raddr.port}",
            "RemoteIP": conn.raddr.ip,
            "ProcessId": conn.pid,
            "ProcessName": process_name,
            "EstadoConexao": conn.status
        })

conn_df = pd.DataFrame(connections)
conn_df.drop_duplicates(subset=["RemoteIP"], inplace=True)
conn_path = f"logs/connections_{timestamp}.csv"
conn_df.to_csv(conn_path, index=False)

# WHOIS + VirusTotal
report = []
for ip in conn_df["RemoteIP"].unique():
    whois_data = "Erro WHOIS"
    vt_result = {"Malicious": 0, "Suspicious": 0, "Harmless": 0}

    try:
        whois_info = IPWhois(ip).lookup_rdap()
        whois_data = whois_info.get("network", {}).get("name", "Desconhecido")
    except:
        pass

    try:
        r = requests.get(VT_URL + ip, headers=HEADERS)
        if r.status_code == 200:
            stats = r.json()['data']['attributes']['last_analysis_stats']
            vt_result = {
                "Malicious": stats.get("malicious", 0),
                "Suspicious": stats.get("suspicious", 0),
                "Harmless": stats.get("harmless", 0)
            }
    except:
        pass

    report.append({"IP": ip, "WHOIS": whois_data, **vt_result})

report_df = pd.DataFrame(report)
report_path = f"logs/report_{timestamp}.csv"
report_df.to_csv(report_path, index=False)

# INTERFACE TKINTER
def create_table(root, df, title):
    label = tk.Label(root, text=title, font=("Arial", 14, "bold"))
    label.pack()
    frame = ttk.Frame(root)
    frame.pack(fill='both', expand=True)
    tree = ttk.Treeview(frame, columns=list(df.columns), show='headings')
    for col in df.columns:
        tree.heading(col, text=col)
        tree.column(col, anchor='center', width=150)
    for _, row in df.iterrows():
        tree.insert('', 'end', values=list(row))
    tree.pack(fill='both', expand=True)

def ver_ips_maliciosos():
    maliciosos = report_df[report_df["Malicious"] > 0]
    if maliciosos.empty:
        messagebox.showinfo("Nenhum IP Malicioso", "Nenhuma atividade maliciosa detectada.")
        return
    for _, row in maliciosos.iterrows():
        ip = row["IP"]
        conn_infos = conn_df[conn_df["RemoteIP"] == ip]
        conexoes = "\n".join([
            f"{r['ProcessName']} ({r['ProcessId']}) - {r['LocalAddress']} → {r['RemoteAddress']} [{r['EstadoConexao']}]"
            for _, r in conn_infos.iterrows()
        ])
        message = (
            f"IP: {ip}\n"
            f"WHOIS: {row['WHOIS']}\n"
            f"Malicious: {row['Malicious']}, Suspicious: {row['Suspicious']}, Harmless: {row['Harmless']}\n\n"
            f"Conexões:\n{conexoes}"
        )
        messagebox.showwarning("IP Potencialmente Malicioso", message)

# Janela Principal
root = tk.Tk()
root.title("Monitoramento de Conexões + WHOIS/VirusTotal")
root.geometry("1500x800")

# Carregar tabelas
conn_df = pd.read_csv(conn_path)
report_df = pd.read_csv(report_path)

# Tabelas
create_table(root, conn_df, "Conexões TCP/UDP")
create_table(root, report_df, "Análise WHOIS e VirusTotal")

# Botão
btn = tk.Button(root, text="Ver IPs Maliciosos", command=ver_ips_maliciosos, bg="red", fg="white", font=("Arial", 12, "bold"))
btn.pack(pady=20)

root.mainloop()


Verificação Whois e Virustotal (total)

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import subprocess
import requests
from ipwhois import IPWhois
import os
import re
from datetime import datetime

# === CONFIGURAÇÕES ===
VT_API_KEY = "your_virustotal_api_key_here"
VT_URL = "https://www.virustotal.com/api/v3/ip_addresses/"
HEADERS = {"x-apikey": VT_API_KEY}
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
os.makedirs("logs", exist_ok=True)

# === FUNÇÕES DE PARSE ===
def get_process_name(pid):
    try:
        output = subprocess.check_output(f'tasklist /FI "PID eq {pid}"', shell=True, text=True)
        lines = output.splitlines()
        if len(lines) > 3:
            return lines[3].split()[0]
    except Exception:
        pass
    return "Desconhecido"

def parse_netstat_output():
    result = subprocess.check_output("netstat -ano", shell=True, universal_newlines=True)
    lines = result.splitlines()
    connections = []

    for line in lines:
        if line.startswith("  "):
            parts = re.split(r"\s+", line.strip())
            if len(parts) >= 5:
                protocolo = parts[0]
                local = parts[1]
                remoto = parts[2]
                estado = parts[3] if protocolo.lower().startswith("tcp") else "LISTENING"
                pid = parts[4] if protocolo.lower().startswith("tcp") else parts[3]

                if ":" not in local or ":" not in remoto:
                    continue  # pula conexões incompletas

                try:
                    local_ip, local_port = re.sub(r"[\[\]]", "", local).rsplit(":", 1)
                    remote_ip, remote_port = re.sub(r"[\[\]]", "", remoto).rsplit(":", 1)
                except ValueError:
                    continue  # pula se falhar

                if remote_ip in ["127.0.0.1", "0.0.0.0", "::1"]:
                    continue  # ignora localhost

                connections.append({
                    "Protocolo": protocolo,
                    "LocalAddress": local,
                    "RemoteAddress": remoto,
                    "RemoteIP": remote_ip,
                    "ProcessId": pid,
                    "ProcessName": get_process_name(pid),
                    "EstadoConexao": estado
                })
    return connections

# === WHOIS + VIRUSTOTAL ===
def enrich_ips(df):
    report = []
    for ip in df["RemoteIP"].unique():
        whois_data = "Erro WHOIS"
        vt_result = {"Malicious": 0, "Suspicious": 0, "Harmless": 0}

        try:
            whois_info = IPWhois(ip).lookup_rdap()
            whois_data = whois_info.get("network", {}).get("name", "Desconhecido")
        except:
            pass

        try:
            r = requests.get(VT_URL + ip, headers=HEADERS)
            if r.status_code == 200:
                stats = r.json()['data']['attributes']['last_analysis_stats']
                vt_result = {
                    "Malicious": stats.get("malicious", 0),
                    "Suspicious": stats.get("suspicious", 0),
                    "Harmless": stats.get("harmless", 0)
                }
        except:
            pass

        report.append({"IP": ip, "WHOIS": whois_data, **vt_result})
    return report

# === INTERFACE TK ===
def create_table(root, df, title):
    label = tk.Label(root, text=title, font=("Arial", 14, "bold"))
    label.pack()
    frame = ttk.Frame(root)
    frame.pack(fill='both', expand=True)
    tree = ttk.Treeview(frame, columns=list(df.columns), show='headings')
    for col in df.columns:
        tree.heading(col, text=col)
        tree.column(col, anchor='center', width=150)
    for _, row in df.iterrows():
        tree.insert('', 'end', values=list(row))
    tree.pack(fill='both', expand=True)

def ver_ips_maliciosos():
    maliciosos = report_df[report_df["Malicious"] > 0]
    if maliciosos.empty:
        messagebox.showinfo("Nenhum IP Malicioso", "Nenhuma atividade maliciosa detectada.")
        return
    for _, row in maliciosos.iterrows():
        ip = row["IP"]
        conn_infos = conn_df[conn_df["RemoteIP"] == ip]
        conexoes = "\n".join([
            f"{r['ProcessName']} ({r['ProcessId']}) - {r['LocalAddress']} → {r['RemoteAddress']} [{r['EstadoConexao']}]"
            for _, r in conn_infos.iterrows()
        ])
        message = (
            f"IP: {ip}\n"
            f"WHOIS: {row['WHOIS']}\n"
            f"Malicious: {row['Malicious']}, Suspicious: {row['Suspicious']}, Harmless: {row['Harmless']}\n\n"
            f"Conexões:\n{conexoes}"
        )
        messagebox.showwarning("IP Potencialmente Malicioso", message)

# === EXECUÇÃO ===
conn_data = parse_netstat_output()
conn_df = pd.DataFrame(conn_data).drop_duplicates(subset=["RemoteIP"])
conn_path = f"logs/connections_{timestamp}.csv"
conn_df.to_csv(conn_path, index=False)

report_data = enrich_ips(conn_df)
report_df = pd.DataFrame(report_data)
report_path = f"logs/report_{timestamp}.csv"
report_df.to_csv(report_path, index=False)

# === TKINTER UI ===
root = tk.Tk()
root.title("Monitoramento de Conexões (sem psutil) + WHOIS/VirusTotal")
root.geometry("1500x800")

create_table(root, conn_df, "Conexões Ativas")
create_table(root, report_df, "Análise WHOIS e VirusTotal")

btn = tk.Button(root, text="Ver IPs Maliciosos", command=ver_ips_maliciosos, bg="red", fg="white", font=("Arial", 12, "bold"))
btn.pack(pady=20)

root.mainloop()


Codigo Completo

In [ ]:
import psutil
import subprocess
import requests
from ipwhois import IPWhois
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import os
from datetime import datetime

# === CONFIGURAÇÕES ===
VT_API_KEY = "your_virustotal_api_key_here"
VT_URL = "https://www.virustotal.com/api/v3/ip_addresses/"
HEADERS = {"x-apikey": VT_API_KEY}
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
os.makedirs("logs", exist_ok=True)

# === FUNÇÕES AUXILIARES ===
def get_svchost_services(pid):
    try:
        output = subprocess.check_output(
            ["powershell", "-Command",
             f"Get-WmiObject -Class Win32_Service | Where-Object {{$_.ProcessId -eq {pid}}} | Select-Object -ExpandProperty Name"],
            universal_newlines=True
        )
        return ", ".join([line.strip() for line in output.strip().splitlines() if line.strip()])
    except Exception:
        return "Nenhum serviço associado"

def get_process_name(pid):
    try:
        proc = psutil.Process(pid)
        return proc.name()
    except (psutil.NoSuchProcess, psutil.AccessDenied):
        return "Desconhecido"

# === COLETA DE CONEXÕES TCP/UDP ===
def get_tcp_connections():
    data = []
    for conn in psutil.net_connections(kind='tcp'):
        try:
            proc = psutil.Process(conn.pid) if conn.pid else None
            pname = proc.name() if proc else "N/A"
            services = get_svchost_services(conn.pid) if pname.lower() == "svchost.exe" else "Nenhum serviço associado"

            row = [
                conn.laddr.ip if conn.laddr else '',
                conn.laddr.port if conn.laddr else '',
                conn.raddr.ip if conn.raddr else '',
                conn.raddr.port if conn.raddr else '',
                conn.pid,
                pname,
                services,
                conn.status
            ]
            data.append(row)
        except Exception:
            continue
    return data

def get_udp_connections():
    data = []
    for conn in psutil.net_connections(kind='udp'):
        try:
            proc = psutil.Process(conn.pid) if conn.pid else None
            pname = proc.name() if proc else "Desconhecido/Finalizado"
            services = get_svchost_services(conn.pid) if pname.lower() == "svchost.exe" else "Nenhum serviço associado"

            row = [
                conn.laddr.ip if conn.laddr else '',
                conn.laddr.port if conn.laddr else '',
                '',
                '',
                conn.pid,
                pname,
                services,
                'UDP'
            ]
            data.append(row)
        except Exception:
            continue
    return data

# === WHOIS + VIRUSTOTAL ===
def enrich_ips(df):
    report = []
    for ip in df["RemoteAddress"].unique():  # Alterado de "RemoteIP" para "RemoteAddress"
        whois_data = "Erro WHOIS"
        vt_result = {"Malicious": 0, "Suspicious": 0, "Harmless": 0}

        try:
            whois_info = IPWhois(ip).lookup_rdap()
            whois_data = whois_info.get("network", {}).get("name", "Desconhecido")
        except:
            pass

        try:
            r = requests.get(VT_URL + ip, headers=HEADERS)
            if r.status_code == 200:
                stats = r.json()['data']['attributes']['last_analysis_stats']
                vt_result = {
                    "Malicious": stats.get("malicious", 0),
                    "Suspicious": stats.get("suspicious", 0),
                    "Harmless": stats.get("harmless", 0)
                }
        except:
            pass

        report.append({"IP": ip, "WHOIS": whois_data, **vt_result})
    return report

# === INTERFACE TK ===
def create_table(root, df, title):
    label = tk.Label(root, text=title, font=("Arial", 14, "bold"))
    label.pack()
    frame = ttk.Frame(root)
    frame.pack(fill='both', expand=True)
    tree = ttk.Treeview(frame, columns=list(df.columns), show='headings')
    for col in df.columns:
        tree.heading(col, text=col)
        tree.column(col, anchor='center', width=150)
    for _, row in df.iterrows():
        tree.insert('', 'end', values=list(row))
    tree.pack(fill='both', expand=True)

def ver_ips_maliciosos():
    maliciosos = report_df[report_df["Malicious"] > 0]
    if maliciosos.empty:
        messagebox.showinfo("Nenhum IP Malicioso", "Nenhuma atividade maliciosa detectada.")
        return
    for _, row in maliciosos.iterrows():
        ip = row["IP"]
        conn_infos = conn_df[conn_df["RemoteAddress"] == ip]
        conexoes = "\n".join([
            f"{r['ProcessName']} ({r['ProcessId']}) - {r['LocalAddress']} → {r['RemoteAddress']} [{r['State']}]"
            for _, r in conn_infos.iterrows()
        ])
        message = (
            f"IP: {ip}\n"
            f"WHOIS: {row['WHOIS']}\n"
            f"Malicious: {row['Malicious']}, Suspicious: {row['Suspicious']}, Harmless: {row['Harmless']}\n\n"
            f"Conexões:\n{conexoes}"
        )
        messagebox.showwarning("IP Potencialmente Malicioso", message)

# === EXECUÇÃO ===
tcp_data = get_tcp_connections()
udp_data = get_udp_connections()
full_data = tcp_data + udp_data
conn_df = pd.DataFrame(full_data, columns=["LocalAddress", "LocalPort", "RemoteAddress", "RemotePort", "ProcessId", "ProcessName", "Services", "State"])

# Cria a coluna "RemoteIP" com os valores de "RemoteAddress"
conn_df["RemoteIP"] = conn_df["RemoteAddress"]

conn_df = conn_df.drop_duplicates(subset=["RemoteAddress"])

conn_path = f"logs/connections_{timestamp}.csv"
conn_df.to_csv(conn_path, index=False)

report_data = enrich_ips(conn_df)
report_df = pd.DataFrame(report_data)
report_path = f"logs/report_{timestamp}.csv"
report_df.to_csv(report_path, index=False)

# === TKINTER UI ===
root = tk.Tk()
root.title("Monitoramento de Conexões TCP/UDP + WHOIS/VirusTotal")
root.geometry("1500x800")

create_table(root, conn_df, "Conexões Ativas")
create_table(root, report_df, "Análise WHOIS e VirusTotal")

btn = tk.Button(root, text="Ver IPs Maliciosos", command=ver_ips_maliciosos, bg="red", fg="white", font=("Arial", 12, "bold"))
btn.pack(pady=20)

root.mainloop()


In [ ]:
import psutil
import subprocess
import requests
from ipwhois import IPWhois
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import os
from datetime import datetime

# === CONFIGURAÇÕES ===
VT_API_KEY = "your_virustotal_api_key_here"
VT_URL = "https://www.virustotal.com/api/v3/ip_addresses/"
HEADERS = {"x-apikey": VT_API_KEY}
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
os.makedirs("logs", exist_ok=True)

# === FUNÇÕES AUXILIARES ===
def get_svchost_services(pid):
    try:
        output = subprocess.check_output(
            ["powershell", "-Command",
             f"Get-WmiObject -Class Win32_Service | Where-Object {{$_.ProcessId -eq {pid}}} | Select-Object -ExpandProperty Name"],
            universal_newlines=True
        )
        return ", ".join([line.strip() for line in output.strip().splitlines() if line.strip()])
    except Exception:
        return "Nenhum serviço associado"

def get_process_name(pid):
    try:
        proc = psutil.Process(pid)
        return proc.name()
    except (psutil.NoSuchProcess, psutil.AccessDenied):
        return "Desconhecido"

# === COLETA DE CONEXÕES TCP/UDP ===
def get_tcp_connections():
    data = []
    for conn in psutil.net_connections(kind='tcp'):
        try:
            proc = psutil.Process(conn.pid) if conn.pid else None
            pname = proc.name() if proc else "N/A"
            services = get_svchost_services(conn.pid) if pname.lower() == "svchost.exe" else "Nenhum serviço associado"

            row = [
                conn.laddr.ip if conn.laddr else '',
                conn.laddr.port if conn.laddr else '',
                conn.raddr.ip if conn.raddr else '',
                conn.raddr.port if conn.raddr else '',
                conn.pid,
                pname,
                services,
                conn.status
            ]
            data.append(row)
        except Exception:
            continue
    return data

def get_udp_connections():
    data = []
    for conn in psutil.net_connections(kind='udp'):
        try:
            proc = psutil.Process(conn.pid) if conn.pid else None
            pname = proc.name() if proc else "Desconhecido/Finalizado"
            services = get_svchost_services(conn.pid) if pname.lower() == "svchost.exe" else "Nenhum serviço associado"

            row = [
                conn.laddr.ip if conn.laddr else '',
                conn.laddr.port if conn.laddr else '',
                '',
                '',
                conn.pid,
                pname,
                services,
                'UDP'
            ]
            data.append(row)
        except Exception:
            continue
    return data

# === WHOIS + VIRUSTOTAL COM MELHORIAS ===
def enrich_ips(df):
    report = []
    for ip in df["RemoteAddress"].unique():
        whois_data = "Erro WHOIS"
        vt_result = {"Malicious": 0, "Suspicious": 0, "Harmless": 0}

        # WHOIS com fallback
        try:
            whois_info = IPWhois(ip).lookup_rdap()
            whois_data = whois_info.get("network", {}).get("name", "Desconhecido")
        except Exception as e:
            print(f"[WHOIS fallback] ipwhois falhou para {ip}: {e}")
            try:
                output = subprocess.check_output(["whois", ip], universal_newlines=True, timeout=5)
                for line in output.splitlines():
                    if "OrgName" in line or "Organization" in line:
                        whois_data = line.split(":", 1)[-1].strip()
                        break
            except Exception as e:
                whois_data = "Erro WHOIS"

        # VirusTotal com verificação de resposta válida
        try:
            r = requests.get(VT_URL + ip, headers=HEADERS)
            if r.status_code == 200 and "last_analysis_stats" in r.json().get("data", {}).get("attributes", {}):
                stats = r.json()['data']['attributes']['last_analysis_stats']
                vt_result = {
                    "Malicious": stats.get("malicious", 0),
                    "Suspicious": stats.get("suspicious", 0),
                    "Harmless": stats.get("harmless", 0)
                }
            else:
                print(f"[VT] Limite ou erro para IP {ip}: status {r.status_code}")
        except Exception as e:
            print(f"[VT] Erro ao consultar VT para {ip}: {e}")

        report.append({"IP": ip, "WHOIS": whois_data, **vt_result})
    return report

# === INTERFACE TK ===
def create_table(root, df, title):
    label = tk.Label(root, text=title, font=("Arial", 14, "bold"))
    label.pack()
    frame = ttk.Frame(root)
    frame.pack(fill='both', expand=True)
    tree = ttk.Treeview(frame, columns=list(df.columns), show='headings')
    for col in df.columns:
        tree.heading(col, text=col)
        tree.column(col, anchor='center', width=150)
    for _, row in df.iterrows():
        tree.insert('', 'end', values=list(row))
    tree.pack(fill='both', expand=True)

def ver_ips_maliciosos():
    maliciosos = report_df[report_df["Malicious"] > 0]
    if maliciosos.empty:
        messagebox.showinfo("Nenhum IP Malicioso", "Nenhuma atividade maliciosa detectada.")
        return
    for _, row in maliciosos.iterrows():
        ip = row["IP"]
        conn_infos = conn_df[conn_df["RemoteAddress"] == ip]
        conexoes = "\n".join([
            f"{r['ProcessName']} ({r['ProcessId']}) - {r['LocalAddress']} → {r['RemoteAddress']} [{r['State']}]"
            for _, r in conn_infos.iterrows()
        ])
        message = (
            f"IP: {ip}\n"
            f"WHOIS: {row['WHOIS']}\n"
            f"Malicious: {row['Malicious']}, Suspicious: {row['Suspicious']}, Harmless: {row['Harmless']}\n\n"
            f"Conexões:\n{conexoes}"
        )
        messagebox.showwarning("IP Potencialmente Malicioso", message)

# === EXECUÇÃO ===
tcp_data = get_tcp_connections()
udp_data = get_udp_connections()
full_data = tcp_data + udp_data
conn_df = pd.DataFrame(full_data, columns=["LocalAddress", "LocalPort", "RemoteAddress", "RemotePort", "ProcessId", "ProcessName", "Services", "State"])
conn_df["RemoteIP"] = conn_df["RemoteAddress"]  # redundância para compatibilidade
conn_df = conn_df.drop_duplicates(subset=["RemoteAddress"])

conn_path = f"logs/connections_{timestamp}.csv"
conn_df.to_csv(conn_path, index=False)

report_data = enrich_ips(conn_df)
report_df = pd.DataFrame(report_data)
report_path = f"logs/report_{timestamp}.csv"
report_df.to_csv(report_path, index=False)

# === TKINTER UI ===
root = tk.Tk()
root.title("Monitoramento de Conexões TCP/UDP + WHOIS/VirusTotal")
root.geometry("1500x800")

create_table(root, conn_df, "Conexões Ativas")
create_table(root, report_df, "Análise WHOIS e VirusTotal")

btn = tk.Button(root, text="Ver IPs Maliciosos", command=ver_ips_maliciosos, bg="red", fg="white", font=("Arial", 12, "bold"))
btn.pack(pady=20)

root.mainloop()


[VT] Limite ou erro para IP 2800:3f0:4001:800::200a: status 429
[WHOIS fallback] ipwhois falhou para 127.0.0.1: IPv4 address 127.0.0.1 is already defined as Loopback via RFC 1122, Section 3.2.1.3.
[VT] Limite ou erro para IP 127.0.0.1: status 429
[VT] Limite ou erro para IP 2800:3f0:4001:815::200e: status 429
[VT] Limite ou erro para IP 2606:4700:4400::6812:2929: status 429
[WHOIS fallback] ipwhois falhou para : '' does not appear to be an IPv4 or IPv6 address
[VT] Limite ou erro para IP : status 404
[VT] Limite ou erro para IP 34.110.207.168: status 429
[VT] Limite ou erro para IP 2800:3f0:4001:83e::200a: status 429
[VT] Limite ou erro para IP 2800:3f0:4001:80b::2005: status 429
[VT] Limite ou erro para IP 3.224.251.111: status 429
[VT] Limite ou erro para IP 2800:3f0:4001:836::2003: status 429
[VT] Limite ou erro para IP 2800:3f0:4001:83c::200a: status 429
[VT] Limite ou erro para IP 157.240.12.54: status 429
[VT] Limite ou erro para IP 35.186.227.140: status 429
[VT] Limite ou erro 